# 作业：手写数字加法机

本文件是与集智AI学园出品的系列课程“火炬上的深度学习”配套的作业notebook。本作业要求学员构造一个卷积神经网，输入两张手写数字图片，输出这两个数字的和。

本文件提供了一个完成做的大框架，学员需要自行修改、添加代码，从而完成任务

本文件是集智AI学园http://campus.swarma.org 出品的“火炬上的深度学习”第III课的配套源代码


In [33]:
# 导入所需要的包，请保证torchvision已经在你的环境中安装好
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [34]:
USE_CUDA = torch.cuda.is_available()

In [35]:

# 定义需要用到的超参数
image_size = 28  #图像的总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 20  #训练的总循环周期
batch_size = 64

# 加载MINIST数据，如果没有下载过，就会在当前路径下新建/data子目录，并把文件存放其中

train_dataset = dsets.MNIST(root='./data',  #文件存放路径
                            train=True,   #提取训练集
                            transform=transforms.ToTensor(),  #将图像转化为Tensor
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# 由于每一个样本需要输入两个图片，因此每一个loader和sampler都有两个

sampler1 = torch.utils.data.sampler.SubsetRandomSampler(
    np.random.permutation(range(len(train_dataset))))
sampler2 = torch.utils.data.sampler.SubsetRandomSampler(
    np.random.permutation(range(len(train_dataset))))

# 训练数据的两个加载器
train_loader1 = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = False,
                                           sampler = sampler1
                                           )
train_loader2 = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = False,
                                           sampler = sampler2
                                           )
#vlueerror: mutually exclussive with shuffle??

# 校验数据和测试数据都各自有两套
val_size = 5000
val_indices1 = range(val_size)
val_indices2 = np.random.permutation(range(val_size))
test_indices1 = range(val_size, len(test_dataset))
test_indices2 = np.random.permutation(test_indices1)
val_sampler1 = torch.utils.data.sampler.SubsetRandomSampler(val_indices1)
val_sampler2 = torch.utils.data.sampler.SubsetRandomSampler(val_indices2)

test_sampler1 = torch.utils.data.sampler.SubsetRandomSampler(test_indices1)
test_sampler2 = torch.utils.data.sampler.SubsetRandomSampler(test_indices2)

val_loader1 = torch.utils.data.DataLoader(dataset = test_dataset,
                                        batch_size = batch_size,
                                        shuffle = False,
                                        sampler = val_sampler1
                                        )
val_loader2 = torch.utils.data.DataLoader(dataset = test_dataset,
                                        batch_size = batch_size,
                                        shuffle = False,
                                        sampler = val_sampler2
                                        )
test_loader1 = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False,
                                         sampler = test_sampler1
                                         )
test_loader2 = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False,
                                         sampler = test_sampler2
                                         )

# MINST Adder

为了实现加法器，需要同时处理两个手写体数字图像，并对它进行相应的图像处理
因此，网络的架构为两个卷积神经网络，串联上两个全链接层

In [72]:
depth = [32, 64]
fc = [1024, 32, 1]
class MINSTAdder(nn.Module):
    def __init__(self):
        super(MINSTAdder, self).__init__()
        #公共网络部件
        self.net_pool = nn.MaxPool2d(kernel_size=2, stride=2) # only hyperparameter here
        self.bc1 = nn.BatchNorm1d(num_features=fc[0]) # batch normalization, because of the batch training
        self.bc2 = nn.BatchNorm1d(num_features=fc[1])
        
        #处理第一个图像处理用的卷积网络部件
        self.net1_conv1 = nn.Conv2d(in_channels=1, out_channels=depth[0], kernel_size=5, padding = 2)
        # feature map size is 14*14 by pooling
        # padding=2 for same padding
        self.net1_conv2 = nn.Conv2d(depth[0], depth[1], 5, padding = 2) 
        # feature map size is 7*7 by pooling
         
        #处理第二个图像处理用的卷积网络部件
        self.net2_conv1 = nn.Conv2d(in_channels=1, out_channels=depth[0], kernel_size=5, padding = 2)
        # feature map size is 14*14 by pooling
        # padding=2 for same padding
        self.net2_conv2 = nn.Conv2d(depth[0], depth[1], 5, padding = 2) 
        # feature map size is 7*7 by pooling      
        
        #后面的全连阶层
        self.fc1 = nn.Linear(in_features= 2 * depth[1] * (image_size // 4) * (image_size // 4), 
                             out_features=fc[0])
        self.fc2 = nn.Linear(fc[0], fc[1])
        self.fc3 = nn.Linear(fc[1], fc[2])
        
    def forward(self, x, y, training = True):
        #第一张图像的处理流程 
        x = F.relu(self.net1_conv1(x))         # 1*28*28 --> 32*28*28
        x = self.net_pool(x)                   # 32*28*28 --> 32*14*14
        x = F.relu(self.net1_conv2(x))         # 32*14*14 --> 64*14*14
        x = self.net_pool(x)                   # 64*14*14 --> 64*7*7
        x = x.view(-1, depth[1] * (image_size // 4) * (image_size // 4)) # flatten, reshape variable
         
        #第二张图像的处理流程
        y = F.relu(self.net2_conv1(y))         # 1*28*28 --> 32*28*28
        y = self.net_pool(y)                   # 32*28*28 --> 32*14*14
        y = F.relu(self.net2_conv2(y))         # 32*14*14 --> 64*14*14
        y = self.net_pool(y)                   # 64*14*14 --> 64*7*7
        y = y.view(-1, depth[1] * (image_size // 4) * (image_size // 4)) # flatten, reshape variable
         
        #将前两部处理得到的张量并列到一起，喂给两层全链接前馈网络，最后输出预测数值
        z = torch.cat((x, y), 1)                # 2,1024 --> 2048 
        z = F.relu(self.bc1(self.fc1(z)))       # 2048 -->1024
        z = F.dropout(z, p=0.5, training=self.training) #以默认为0.5的概率对这一层进行dropout操作
        z = F.relu(self.bc2(self.fc2(z)))       # 1024 --> 32
        z = F.dropout(z, p=0.2, training=self.training)
        z = F.relu(self.fc3(z))                 # 32 --> 1    
        return z
        

# 计算准确度的函数（有多少数字给出了严格的正确输出结果）
def rightness(y, target):
    out = torch.round(y.view(-1)).type(torch.LongTensor)
    out = out.eq(target).sum()
    out1 = y.size()[0]
    return(out, out1)
net = MINSTAdder()
net

MINSTAdder (
  (net_pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (bc1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True)
  (bc2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True)
  (net1_conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (net1_conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (net2_conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (net2_conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear (6272 -> 1024)
  (fc2): Linear (1024 -> 32)
  (fc3): Linear (32 -> 1)
)

In [73]:
#将网络定义为一个预测器，来对加法的结果进行预测，因此用MSE平均平房误差作为我们的损失函数
net.cuda()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr = 0.0001, momentum = 0.9)
results = {}

In [74]:
%%time
# 开始训练循环，本部分代码需要补齐
records = []
for epoch in range(num_epochs):
    losses = []
    # 一个关键技术难点是对两个数据加载器进行读取成对儿的数据。我们的办法是通过zip命令，将loader1和2并列在一起，一对一对的读取数据
    for idx, data in enumerate(zip(train_loader1, train_loader2)):
        ((x1, y1), (x2, y2)) = data
        optimizer.zero_grad()
        net.train()
        outputs = net(Variable(x1).cuda(), Variable(x2).cuda())
        labels = y1 + y2
        loss = criterion(outputs, Variable(labels.type(torch.FloatTensor)).cuda())
        loss.backward()
        optimizer.step()
        losses.append(loss.data[0])
#       right = rightness(output, target)
#       losses.append((right[1]-right[0]) / right[1] * 1.0) # (总样例数-正确样例数)/总样例数
        if idx % 100 == 0:
            net.eval()
            val_losses = []
            val_rights = []
            for ((x1,y1),(x2, y2)) in zip(val_loader1,val_loader2):
                out = net(Variable(x1).cuda(), Variable(x2).cuda())
                labels = y1 + y2
                loss = criterion(out, Variable(labels.type(torch.FloatTensor)).cuda())
                val_losses.append(loss.data[0])
                right = rightness(out, Variable(labels))
                val_rights.append(right)
            val_r = (sum([tup[0].data[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
#           print(val_r)
            right_ratio = 100.0 * val_r[0] / val_r[1]
            
            #每间隔一定周期就打印一下训练集、校验集的准确率结果
            print('第{}周期，第({}/{})个撮，训练误差：{:.2f}, 校验误差：{:.2f}, 准确率：{:.2f}'.format(
                epoch, idx, len(train_loader1),
                np.mean(losses), np.mean(val_losses), right_ratio))
            records.append([np.mean(losses), np.mean(val_losses), right_ratio])

第0周期，第(0/938)个撮，训练误差：91.12, 校验误差：97.02, 准确率：0.82
第0周期，第(100/938)个撮，训练误差：76.86, 校验误差：39.90, 准确率：2.80
第0周期，第(200/938)个撮，训练误差：47.86, 校验误差：8.04, 准确率：12.94
第0周期，第(300/938)个撮，训练误差：34.38, 校验误差：4.14, 准确率：21.48
第0周期，第(400/938)个撮，训练误差：27.32, 校验误差：3.37, 准确率：24.74
第0周期，第(500/938)个撮，训练误差：22.94, 校验误差：3.06, 准确率：26.62
第0周期，第(600/938)个撮，训练误差：19.97, 校验误差：3.26, 准确率：26.70
第0周期，第(700/938)个撮，训练误差：17.82, 校验误差：3.12, 准确率：27.20
第0周期，第(800/938)个撮，训练误差：16.15, 校验误差：2.89, 准确率：28.56
第0周期，第(900/938)个撮，训练误差：14.82, 校验误差：2.56, 准确率：30.82
第1周期，第(0/938)个撮，训练误差：3.57, 校验误差：3.55, 准确率：23.44
第1周期，第(100/938)个撮，训练误差：4.15, 校验误差：2.46, 准确率：30.50
第1周期，第(200/938)个撮，训练误差：4.01, 校验误差：2.68, 准确率：29.92
第1周期，第(300/938)个撮，训练误差：3.93, 校验误差：2.31, 准确率：33.20
第1周期，第(400/938)个撮，训练误差：3.89, 校验误差：2.30, 准确率：33.32
第1周期，第(500/938)个撮，训练误差：3.84, 校验误差：2.45, 准确率：30.04
第1周期，第(600/938)个撮，训练误差：3.78, 校验误差：3.03, 准确率：25.56
第1周期，第(700/938)个撮，训练误差：3.74, 校验误差：2.28, 准确率：32.66
第1周期，第(800/938)个撮，训练误差：3.68, 校验误差：2.60, 准确率：29.62
第1周期，第(900/938)个撮，训练误差：3.64, 校验误差：1.93, 准确率：

第16周期，第(400/938)个撮，训练误差：1.44, 校验误差：0.81, 准确率：62.22
第16周期，第(500/938)个撮，训练误差：1.41, 校验误差：0.85, 准确率：61.82
第16周期，第(600/938)个撮，训练误差：1.41, 校验误差：0.82, 准确率：62.12
第16周期，第(700/938)个撮，训练误差：1.41, 校验误差：0.79, 准确率：61.96
第16周期，第(800/938)个撮，训练误差：1.41, 校验误差：0.81, 准确率：63.84
第16周期，第(900/938)个撮，训练误差：1.41, 校验误差：0.86, 准确率：61.54
第17周期，第(0/938)个撮，训练误差：1.65, 校验误差：0.79, 准确率：63.52
第17周期，第(100/938)个撮，训练误差：1.32, 校验误差：0.79, 准确率：63.24
第17周期，第(200/938)个撮，训练误差：1.35, 校验误差：0.84, 准确率：63.34
第17周期，第(300/938)个撮，训练误差：1.38, 校验误差：0.76, 准确率：64.38
第17周期，第(400/938)个撮，训练误差：1.40, 校验误差：0.78, 准确率：64.40
第17周期，第(500/938)个撮，训练误差：1.41, 校验误差：0.81, 准确率：64.74
第17周期，第(600/938)个撮，训练误差：1.41, 校验误差：0.81, 准确率：63.36
第17周期，第(700/938)个撮，训练误差：1.42, 校验误差：0.80, 准确率：65.12
第17周期，第(800/938)个撮，训练误差：1.42, 校验误差：0.80, 准确率：64.46
第17周期，第(900/938)个撮，训练误差：1.41, 校验误差：0.79, 准确率：63.52
第18周期，第(0/938)个撮，训练误差：1.45, 校验误差：0.84, 准确率：59.96
第18周期，第(100/938)个撮，训练误差：1.39, 校验误差：0.77, 准确率：63.26
第18周期，第(200/938)个撮，训练误差：1.35, 校验误差：0.84, 准确率：63.14
第18周期，第(300/938)个撮，训练误差：1.37, 校验误差：

In [78]:
# 在测试集上运行我们的加法机网络，并测试预测准确度
torch.save(net, 'net_adder.pkl')
# 在测试集上运行我们的加法机网络，并测试预测准确度
net.eval()
vals = []
for ((x1,y1),(x2, y2)) in zip(test_loader1,test_loader2):
    net.cpu()
    out = net(Variable(x1), Variable(x2))
    label = y1 + y2
    right = rightness(out, Variable(label))
    vals.append(right)
    
#计算准确率
val_r = (sum([tup[0].data[0] for tup in vals]), sum([tup[1] for tup in vals]))
right_rate = 1.0 * val_r[0] / val_r[1]
right_rate



/home/ken/anaconda3/envs/tf12/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type MINSTAdder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


0.7546